### Importar bibliotecas

In [1]:
import pandas as pd
from joblib import Parallel, delayed
import multiprocessing

##### Ler os dados das ações

In [2]:
dados = pd.read_csv('../dados/historico_acoes.csv', encoding='latin-1')

dados.head()

/root/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,TIPO,DATA,CODBDI,CODNEG,TPMERC,NOMRES,ESPECI,PRAZOT,MODREF,PREABE,...,TOTNEG,QUATOT,VOLTOT,PREEXE,INDOPC,DATVEN,FATCOT,PTOEXE,CODISI,DISMES
0,1,19880104,2.0,ABC 6,10,ABC XTAL,PPA,NaN,CZ$,225.0,...,2.0,3000.0,685000.0,0.0,0.0,99991231.0,1.0,0.0,ABCXACPPA,101.0
1,1,19880104,2.0,ACE 2,10,ACESITA,PP C01,NaN,CZ$,440.0,...,1.0,400.0,176000.0,0.0,0.0,99991231.0,1.0,0.0,ACESACPP,114.0
2,1,19880104,2.0,EAL 2,10,ACO ALTONA,PP,NaN,CZ$,863.0,...,8.0,33100.0,29863800.0,0.0,0.0,99991231.0,1.0,0.0,EALTACPP,117.0
3,1,19880104,2.0,AVI 2,10,ACOS VILL,PP C42,NaN,CZ$,330.0,...,12.0,112700.0,37149000.0,0.0,0.0,99991231.0,1.0,0.0,AVILACPP,140.0
4,1,19880104,2.0,CRA 2,10,ADUBOS CRA,PP C31,NaN,CZ$,90.0,...,1.0,10000.0,900000.0,0.0,0.0,99991231.0,1.0,0.0,CRADACPP,121.0


##### Filtrando apenas as coluncas úteis

In [3]:
dados = dados[['DATA', 'CODNEG', 'PREABE', 'PREULT']]

dados.head()

,DATA,CODNEG,PREABE,PREULT
0,19880104,ABC 6,225.0,230.0
1,19880104,ACE 2,440.0,440.0
2,19880104,EAL 2,863.0,920.0
3,19880104,AVI 2,330.0,330.0
4,19880104,CRA 2,90.0,90.0


### Criar váriavel classe

A váriavel classe representa se a ação teve desempenho positivo ou negativo no dia.

Desempenho positivo: O valor final da ação no dia fechou maior ou igual ao valor de abertura

Desempenho negativo: Se o valor de fechamento for menor que o valor de abertura, entao a ação teve diminuição/queda, logo desempenho negativo

In [4]:
# Criar váriavel classe
dados['STATUS_POSITIVO'] = 1

# Se o valor de fechamento for menor que o valor de abertura, entao a ação teve diminuição/queda
dados.loc[dados['PREULT']<dados['PREABE'],'STATUS_POSITIVO'] = -1

In [5]:
dados['STATUS_POSITIVO'].unique()

array([ 1, -1])

##### Verificando os dados das ações PETR4 e VALE3

In [6]:
dados.loc[dados['CODNEG']=='PETR4'].head()

,DATA,CODNEG,PREABE,PREULT,STATUS_POSITIVO
168517,20080102,PETR4,8699.0,8700.0,1
169690,20080103,PETR4,8670.0,8560.0,-1
170909,20080104,PETR4,8565.0,8140.0,-1
172103,20080107,PETR4,8146.0,7901.0,-1
173318,20080108,PETR4,8030.0,8335.0,1


In [7]:
dados.loc[dados['CODNEG']=='VALE3'].head()

,DATA,CODNEG,PREABE,PREULT,STATUS_POSITIVO
168896,20080102,VALE3,5925.0,5790.0,-1
170049,20080103,VALE3,5806.0,5820.0,1
171307,20080104,VALE3,5870.0,5610.0,-1
172460,20080107,VALE3,5648.0,5510.0,-1
173717,20080108,VALE3,5600.0,5620.0,1


***
### Lendo dimensão data

In [8]:
dimensao_data = pd.read_csv('../dados/dimensao_data.csv')

dimensao_data.head()

,date_dim_id,date_actual,epoch,day_suffix,day_name,day_of_week,day_of_month,day_of_quarter,day_of_year,week_of_month,...,last_day_of_week,first_day_of_month,last_day_of_month,first_day_of_quarter,last_day_of_quarter,first_day_of_year,last_day_of_year,mmyyyy,mmddyyyy,weekend_indr
0,19700101,1970-01-01,0,5th,Thursday,4,1,1,1,1,...,1970-01-04,1970-01-01,1970-01-31,1970-01-01,1970-03-31,1970-01-01,1970-12-31,11970,1011970,False
1,19700102,1970-01-02,86400,6th,Friday,5,2,2,2,1,...,1970-01-04,1970-01-01,1970-01-31,1970-01-01,1970-03-31,1970-01-01,1970-12-31,11970,1021970,False
2,19700103,1970-01-03,172800,7th,Saturday,6,3,3,3,1,...,1970-01-04,1970-01-01,1970-01-31,1970-01-01,1970-03-31,1970-01-01,1970-12-31,11970,1031970,True
3,19700104,1970-01-04,259200,1st,Sunday,7,4,4,4,1,...,1970-01-04,1970-01-01,1970-01-31,1970-01-01,1970-03-31,1970-01-01,1970-12-31,11970,1041970,True
4,19700105,1970-01-05,345600,2nd,Monday,1,5,5,5,1,...,1970-01-11,1970-01-01,1970-01-31,1970-01-01,1970-03-31,1970-01-01,1970-12-31,11970,1051970,False


In [9]:
dimensao_data.tail()

,date_dim_id,date_actual,epoch,day_suffix,day_name,day_of_week,day_of_month,day_of_quarter,day_of_year,week_of_month,...,last_day_of_week,first_day_of_month,last_day_of_month,first_day_of_quarter,last_day_of_quarter,first_day_of_year,last_day_of_year,mmyyyy,mmddyyyy,weekend_indr
29215,20491227,2049-12-27,2524176000,2nd,Monday,1,27,88,361,4,...,2050-01-02,2049-12-01,2049-12-31,2049-10-01,2049-12-31,2049-01-01,2049-12-31,122049,12272049,False
29216,20491228,2049-12-28,2524262400,3rd,Tuesday,2,28,89,362,4,...,2050-01-02,2049-12-01,2049-12-31,2049-10-01,2049-12-31,2049-01-01,2049-12-31,122049,12282049,False
29217,20491229,2049-12-29,2524348800,4th,Wednesday,3,29,90,363,5,...,2050-01-02,2049-12-01,2049-12-31,2049-10-01,2049-12-31,2049-01-01,2049-12-31,122049,12292049,False
29218,20491230,2049-12-30,2524435200,5th,Thursday,4,30,91,364,5,...,2050-01-02,2049-12-01,2049-12-31,2049-10-01,2049-12-31,2049-01-01,2049-12-31,122049,12302049,False
29219,20491231,2049-12-31,2524521600,6th,Friday,5,31,92,365,5,...,2050-01-02,2049-12-01,2049-12-31,2049-10-01,2049-12-31,2049-01-01,2049-12-31,122049,12312049,False


##### Renomeando coluna "key" para unir com os dados das ações

In [10]:
dimensao_data=dimensao_data.rename(columns = {'date_dim_id':'DATA'})

dimensao_data.head()

,DATA,date_actual,epoch,day_suffix,day_name,day_of_week,day_of_month,day_of_quarter,day_of_year,week_of_month,...,last_day_of_week,first_day_of_month,last_day_of_month,first_day_of_quarter,last_day_of_quarter,first_day_of_year,last_day_of_year,mmyyyy,mmddyyyy,weekend_indr
0,19700101,1970-01-01,0,5th,Thursday,4,1,1,1,1,...,1970-01-04,1970-01-01,1970-01-31,1970-01-01,1970-03-31,1970-01-01,1970-12-31,11970,1011970,False
1,19700102,1970-01-02,86400,6th,Friday,5,2,2,2,1,...,1970-01-04,1970-01-01,1970-01-31,1970-01-01,1970-03-31,1970-01-01,1970-12-31,11970,1021970,False
2,19700103,1970-01-03,172800,7th,Saturday,6,3,3,3,1,...,1970-01-04,1970-01-01,1970-01-31,1970-01-01,1970-03-31,1970-01-01,1970-12-31,11970,1031970,True
3,19700104,1970-01-04,259200,1st,Sunday,7,4,4,4,1,...,1970-01-04,1970-01-01,1970-01-31,1970-01-01,1970-03-31,1970-01-01,1970-12-31,11970,1041970,True
4,19700105,1970-01-05,345600,2nd,Monday,1,5,5,5,1,...,1970-01-11,1970-01-01,1970-01-31,1970-01-01,1970-03-31,1970-01-01,1970-12-31,11970,1051970,False


In [11]:
dimensao_data.dtypes

DATA                       int64
date_actual               object
epoch                      int64
day_suffix                object
day_name                  object
day_of_week                int64
day_of_month               int64
day_of_quarter             int64
day_of_year                int64
week_of_month              int64
week_of_year               int64
week_of_year_iso          object
month_actual               int64
month_name                object
month_name_abbreviated    object
quarter_actual             int64
quarter_name              object
year_actual                int64
first_day_of_week         object
last_day_of_week          object
first_day_of_month        object
last_day_of_month         object
first_day_of_quarter      object
last_day_of_quarter       object
first_day_of_year         object
last_day_of_year          object
mmyyyy                     int64
mmddyyyy                   int64
weekend_indr                bool
dtype: object

##### Convertendo as datas que estão no formato YYYY-MM-DD para YYYYMMDD, 
##### Essa conversão é necessária para esses atributos ficarem do tipo int, pois com o "-" o atributo fica do tipo object, o que prejudica o desempenho das técnicas

In [12]:
dimensao_data['first_day_of_week'] = dimensao_data['first_day_of_week'].str.replace('-','').apply(int)
dimensao_data['last_day_of_week'] = dimensao_data['last_day_of_week'].str.replace('-','').apply(int)
dimensao_data['first_day_of_month'] = dimensao_data['first_day_of_month'].str.replace('-','').apply(int)
dimensao_data['last_day_of_month'] = dimensao_data['last_day_of_month'].str.replace('-','').apply(int)
dimensao_data['first_day_of_quarter'] = dimensao_data['first_day_of_quarter'].str.replace('-','').apply(int)
dimensao_data['last_day_of_quarter'] = dimensao_data['last_day_of_quarter'].str.replace('-','').apply(int)
dimensao_data['first_day_of_year'] = dimensao_data['first_day_of_year'].str.replace('-','').apply(int)
dimensao_data['last_day_of_year'] = dimensao_data['last_day_of_year'].str.replace('-','').apply(int)

dimensao_data.dtypes

DATA                       int64
date_actual               object
epoch                      int64
day_suffix                object
day_name                  object
day_of_week                int64
day_of_month               int64
day_of_quarter             int64
day_of_year                int64
week_of_month              int64
week_of_year               int64
week_of_year_iso          object
month_actual               int64
month_name                object
month_name_abbreviated    object
quarter_actual             int64
quarter_name              object
year_actual                int64
first_day_of_week          int64
last_day_of_week           int64
first_day_of_month         int64
last_day_of_month          int64
first_day_of_quarter       int64
last_day_of_quarter        int64
first_day_of_year          int64
last_day_of_year           int64
mmyyyy                     int64
mmddyyyy                   int64
weekend_indr                bool
dtype: object

##### Selecionando dados da dimensão data

In [13]:
dimensao_data_selecionados = dimensao_data[['DATA', 'epoch', 'day_of_week', 'day_of_month', 'day_of_quarter', 'day_of_year', 
                                            'week_of_month', 'week_of_year', 'month_actual', 'quarter_actual', 'year_actual',
                                            'first_day_of_week', 'last_day_of_week', 'first_day_of_month', 'last_day_of_month',
                                            'first_day_of_quarter', 'last_day_of_quarter', 'first_day_of_year', 'last_day_of_year'
                                            ]]

dimensao_data_selecionados.tail()

,DATA,epoch,day_of_week,day_of_month,day_of_quarter,day_of_year,week_of_month,week_of_year,month_actual,quarter_actual,year_actual,first_day_of_week,last_day_of_week,first_day_of_month,last_day_of_month,first_day_of_quarter,last_day_of_quarter,first_day_of_year,last_day_of_year
29215,20491227,2524176000,1,27,88,361,4,52,12,4,2049,20491227,20500102,20491201,20491231,20491001,20491231,20490101,20491231
29216,20491228,2524262400,2,28,89,362,4,52,12,4,2049,20491227,20500102,20491201,20491231,20491001,20491231,20490101,20491231
29217,20491229,2524348800,3,29,90,363,5,52,12,4,2049,20491227,20500102,20491201,20491231,20491001,20491231,20490101,20491231
29218,20491230,2524435200,4,30,91,364,5,52,12,4,2049,20491227,20500102,20491201,20491231,20491001,20491231,20490101,20491231
29219,20491231,2524521600,5,31,92,365,5,52,12,4,2049,20491227,20500102,20491201,20491231,20491001,20491231,20490101,20491231


##### Selecionando dados das ações para realizar a união com a dimensão data

In [14]:
dados_selecionados = dados[['DATA', 'CODNEG', 'STATUS_POSITIVO']]

dados_selecionados.head()

,DATA,CODNEG,STATUS_POSITIVO
0,19880104,ABC 6,1
1,19880104,ACE 2,1
2,19880104,EAL 2,1
3,19880104,AVI 2,1
4,19880104,CRA 2,1


***
### Unindo dimensão data com os dados

In [15]:
completo = pd.merge(dados_selecionados, dimensao_data_selecionados, on='DATA')

completo.head()

,DATA,CODNEG,STATUS_POSITIVO,epoch,day_of_week,day_of_month,day_of_quarter,day_of_year,week_of_month,week_of_year,...,quarter_actual,year_actual,first_day_of_week,last_day_of_week,first_day_of_month,last_day_of_month,first_day_of_quarter,last_day_of_quarter,first_day_of_year,last_day_of_year
0,19880104,ABC 6,1,568252800,1,4,4,4,1,1,...,1,1988,19880104,19880110,19880101,19880131,19880101,19880331,19880101,19881231
1,19880104,ACE 2,1,568252800,1,4,4,4,1,1,...,1,1988,19880104,19880110,19880101,19880131,19880101,19880331,19880101,19881231
2,19880104,EAL 2,1,568252800,1,4,4,4,1,1,...,1,1988,19880104,19880110,19880101,19880131,19880101,19880331,19880101,19881231
3,19880104,AVI 2,1,568252800,1,4,4,4,1,1,...,1,1988,19880104,19880110,19880101,19880131,19880101,19880331,19880101,19881231
4,19880104,CRA 2,1,568252800,1,4,4,4,1,1,...,1,1988,19880104,19880110,19880101,19880131,19880101,19880331,19880101,19881231


In [16]:
completo.dtypes

DATA                     int64
CODNEG                  object
STATUS_POSITIVO          int64
epoch                    int64
day_of_week              int64
day_of_month             int64
day_of_quarter           int64
day_of_year              int64
week_of_month            int64
week_of_year             int64
month_actual             int64
quarter_actual           int64
year_actual              int64
first_day_of_week        int64
last_day_of_week         int64
first_day_of_month       int64
last_day_of_month        int64
first_day_of_quarter     int64
last_day_of_quarter      int64
first_day_of_year        int64
last_day_of_year         int64
dtype: object

##### Quantidade de tuplas:

In [17]:
len(completo)

7001521

##### Verificando se existe algum valor nulo nos dados

In [18]:
completo.isnull().values.any()

False

##### Salvando os dados

In [19]:
completo.to_csv('../dados/dados_tratados.csv', index=False)

***
### Filtro: Manter os dados referentes apenas das ações que estão no mercado atualmente

Ações que já encerraram sua participação no mercado de ações não nós interessa.

##### Verificando qual último dia presente nos dados

In [20]:
ultimo_dia = completo.loc[completo['DATA'].astype(str).str[:4] == '2018', 'DATA'].unique()
ultimo_dia = ultimo_dia[-1:]
ultimo_dia

array([20180530])

##### Verificando quais ações operaram no último dia

In [21]:
acoes_no_ultimo_dia = completo.loc[completo['DATA'] == int(ultimo_dia), 'CODNEG'].unique()
acoes_no_ultimo_dia

array(['AALR3', 'AALR3F', 'AAPL34', ..., 'XPML11', 'XPOM11', 'XTED11'], dtype=object)

##### Quantidade de ações no último dia

In [22]:
len(acoes_no_ultimo_dia)

2140

##### Selecionando os dados apenas das ações atuais

In [23]:
dados_atuais = completo.loc[completo['CODNEG'].isin(acoes_no_ultimo_dia)]
dados_atuais.head()

,DATA,CODNEG,STATUS_POSITIVO,epoch,day_of_week,day_of_month,day_of_quarter,day_of_year,week_of_month,week_of_year,...,quarter_actual,year_actual,first_day_of_week,last_day_of_week,first_day_of_month,last_day_of_month,first_day_of_quarter,last_day_of_quarter,first_day_of_year,last_day_of_year
167674,20080102,ABCB4,-1,1199232000,3,2,2,2,1,1,...,1,2008,20071231,20080106,20080101,20080131,20080101,20080331,20080101,20081231
167675,20080102,ABCB4F,1,1199232000,3,2,2,2,1,1,...,1,2008,20071231,20080106,20080101,20080131,20080101,20080331,20080101,20081231
167694,20080102,AELP3,1,1199232000,3,2,2,2,1,1,...,1,2008,20071231,20080106,20080101,20080131,20080101,20080331,20080101,20081231
167701,20080102,AGRO3,1,1199232000,3,2,2,2,1,1,...,1,2008,20071231,20080106,20080101,20080131,20080101,20080331,20080101,20081231
167714,20080102,ALPA4,-1,1199232000,3,2,2,2,1,1,...,1,2008,20071231,20080106,20080101,20080131,20080101,20080331,20080101,20081231


#### Removendo ações que existe somente 1 valor para o atributo classe ou que tenham menos de 30 tuplas

- Remover ações que o atributo 'STATUS_POSITIVO' só tem 1 valor, somente positivo (1), ou somente negativo (0)

- Remover ações que tenham menos do que 30 tuplas de dados

In [24]:
# Coletando as ações
acoes = dados_atuais.CODNEG.unique()

##### Função para verificar se a ação será removida ou não, com base no que foi definido acima

In [25]:
def verificarAcao(acao):
    classes = dados_atuais.loc[dados_atuais['CODNEG'] == acao, 'STATUS_POSITIVO'].unique()
    tuplas = dados_atuais.loc[dados_atuais['CODNEG'] == acao]
    if (len(classes) == 1) or (len(tuplas) < 30):
        print(acao)
        return acao        

##### Realizar a verificação de cada ação, de forma paralela, executando em todos os núcleos do processador

In [26]:
remover_acoes = Parallel(n_jobs=multiprocessing.cpu_count())(delayed(verificarAcao)(acao) for acao in acoes)

PETRR38
IBOVT69
GGBRR76
IBOVH80
IBOVT72
ITSAG13
BBASG1
IBOVH77
IBOVX33
PETRR37
BRKMF18
EMBRG25
GOAUG68
BBDCH47
IBOVR76
DWDP34
IBOVF81
IBOVR74
BBASR299
BBASR334
BOVAS75
IBOVR75
IBOVF82
ITUBG497
ITUBS449
VALEF384
VALEG529
BBASG36
GGBRF152
GGBRR135
PETRF195
PETRR152
VALEG469
ABEVG225
BBDCF356
IBOVT74
ITUBF443
ITUBF493
ITUBR443
PETRX155
BBASS31
BBDCR324
BBDCR71
BBDCS25
IBOVT33
IBOVT75
BBASR302
EVENF58
GOVE11F
ITUBF488
PETRG195
PETRS172
ITUBX409
PETRH202
PETRH205
PETRS192
VALER404
PETRG212
VALEG509
VALER387
PETRH217
VALER427
PETRX182
BOVAR72
EMBRF22
ITUBG449
ITUBN500
PETRR79
IBOVV75
PETRF217
BOVAS65
ITUBF505
ITUBH17
BBDCF73
BBDCR369
WEGES228
ABEVS91
PETRR172
VALEG519
VALES411
IBOVH84
IBOVH87
GOAUJ92
IBOVH89
ABEVR207
VALER472
IBOVH97
IBOVT76
LAMEF171
LAMEF184
VALES424
VALES469
CMIGF889
IBOVT85
PETRG237
PETRS185
ITUBR495
BBDCL379
BRFSR22
ITSAR108
BBASF332
ITUBR480
VALEF414
GOAUD9
VALEF442
IBOVH92
AZULF355
BOVAS74
PETRR217
VALER417
BBASR439
LRENR302
IBOVJ99
BBDCF346
BBSEF305
CMIGR759
ITUBF450


##### Removendo valores nulos da lista de ações a serem removidas

In [27]:
remover_acoes = list(filter(None, remover_acoes))

##### Quantidade de ações a serem removidas

In [28]:
len(remover_acoes)

903

##### Removendo ações

In [29]:
dados_atuais = dados_atuais.loc[~dados_atuais['CODNEG'].isin(remover_acoes)]

dados_atuais.head()

,DATA,CODNEG,STATUS_POSITIVO,epoch,day_of_week,day_of_month,day_of_quarter,day_of_year,week_of_month,week_of_year,...,quarter_actual,year_actual,first_day_of_week,last_day_of_week,first_day_of_month,last_day_of_month,first_day_of_quarter,last_day_of_quarter,first_day_of_year,last_day_of_year
167674,20080102,ABCB4,-1,1199232000,3,2,2,2,1,1,...,1,2008,20071231,20080106,20080101,20080131,20080101,20080331,20080101,20081231
167675,20080102,ABCB4F,1,1199232000,3,2,2,2,1,1,...,1,2008,20071231,20080106,20080101,20080131,20080101,20080331,20080101,20081231
167694,20080102,AELP3,1,1199232000,3,2,2,2,1,1,...,1,2008,20071231,20080106,20080101,20080131,20080101,20080331,20080101,20081231
167701,20080102,AGRO3,1,1199232000,3,2,2,2,1,1,...,1,2008,20071231,20080106,20080101,20080131,20080101,20080331,20080101,20081231
167714,20080102,ALPA4,-1,1199232000,3,2,2,2,1,1,...,1,2008,20071231,20080106,20080101,20080131,20080101,20080331,20080101,20081231


##### Total de ações restante no dataset

In [30]:
len(dados_atuais.CODNEG.unique())

1237

##### Verificando quantidade de tuplas

In [31]:
len(dados_atuais)

1765513

#### O conjunto de dados diminuiu de **7 milhões** de tuplas para **1 milhão e 700 mil** tuplas aproximadamente

***
Salvando os dados tratados

In [32]:
dados_atuais.to_csv('../dados/dados_tratados_acoes_atuais.csv', index=False)